In [1]:
import sys
import py_compile
import pandas as pd
import xgboost as xgb
import numpy as np

sys.path.append(r"../../")
py_compile.compile("../../utility.py")
from utility import rmse
from sklearn.model_selection import train_test_split 

In [2]:
exclude = ['PC', 'SC-NA', 'SST-Indian', 'AO', 'NAO', 'PNA', 'nina3.4', 'nina1+2', 'nina3', 'nina4', 'SIC']

df = pd.read_csv("../../feature-prepare/input-PC3.csv")

features = [feature for feature in list(df.keys()) if feature not in exclude]
target = ['PC']

predPC = []
obsPC = []

idx = [line for line in range(0,32)]
idxx = [line for line in range(32,40)]
train_df = df.iloc[idx][features]
train_target = df.iloc[idx][target]
test_df = df.iloc[idxx][features]
test_target = df.iloc[idxx][target]
    
param = {'nthread':6, #when use hyperthread, xgboost may become slower
              'learning_rate': 0.01, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 1, #5
              'silent': True,
              'subsample': 0.3,
              'colsample_bytree': 1.0,
              'n_estimators': 3000}

dtrain = xgb.DMatrix(train_df, label=train_target, feature_names=features)
dtest  = xgb.DMatrix(test_df, label=test_target, feature_names=features)
    
    
    
cv_res= xgb.cv(param,dtrain,num_boost_round=6000,early_stopping_rounds=3000,nfold=4, metrics='rmse',show_stdv=False)

bst = xgb.train(param,dtrain,num_boost_round=cv_res.shape[0])
pred = bst.predict(dtest)
    
print(pred)

oo = pd.DataFrame(pred)
oo.to_csv('PC3.txt',header=0,index=0)

[12:33:48] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:33:48] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:33:48] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you 

In [3]:
for impt_type in 'weight', 'gain', 'cover', 'total_gain', 'total_cover':
    print(impt_type + ":        " + str(bst.get_score(importance_type=impt_type)))

weight:        {'SST-Pacific': 106, 'SC-EU': 411, 'SC-TP': 256, 'SST-Atlantic': 211, 'Z70': 120}
gain:        {'SST-Pacific': 608560.7865783019, 'SC-EU': 248422.2028212506, 'SC-TP': 561345.3828598323, 'SST-Atlantic': 532916.2985081513, 'Z70': 710189.51553}
cover:        {'SST-Pacific': 7.084905660377358, 'SC-EU': 4.583941605839416, 'SC-TP': 6.69921875, 'SST-Atlantic': 6.9052132701421804, 'Z70': 8.175}
total_gain:        {'SST-Pacific': 64507443.3773, 'SC-EU': 102101525.359534, 'SC-TP': 143704418.01211706, 'SST-Atlantic': 112445338.98521993, 'Z70': 85222741.8636}
total_cover:        {'SST-Pacific': 751.0, 'SC-EU': 1884.0, 'SC-TP': 1715.0, 'SST-Atlantic': 1457.0, 'Z70': 981.0}
